<a href="https://colab.research.google.com/github/djonathank/djonathank/blob/main/Projeto_SOSZap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import unicodedata

def remover_acentos(txt):
    return ''.join(
        c for c in unicodedata.normalize('NFD', txt)
        if unicodedata.category(c) != 'Mn'
    ).lower()

def pergunta_sim_nao(pergunta):
    respostas_sim = {'sim', 's', 'yes', 'y', 'claro', 'ok', 'okay', 'confirmar'}
    respostas_nao = {'não', 'nao', 'n', 'no', 'cancelar', 'negativo'}

    while True:
        resposta = input(f"{pergunta} (sim/não): ").strip().lower()
        if resposta in respostas_sim:
            return True
        elif resposta in respostas_nao:
            return False
        else:
            print("Resposta inválida. Por favor, digite 'sim' ou 'não'.")

def classificar_emergencia(mensagem):
    prompt = f'''
    Você é um atendente de emergência via WhatsApp.

    Analise a seguinte mensagem e classifique como:
    - POLÍCIA → Se for caso de violência, assalto, ameaça, etc.
    - SAMU → Se for caso médico: desmaio, acidente, sangramento, etc.

    Formato da resposta (sem explicações, apenas isso):
    Tipo: POLÍCIA ou SAMU
    Resposta: mensagem empática e clara para o solicitante

    Mensagem: "{mensagem}"
    '''

    # Simulação de resposta do modelo (substituir pela chamada real do Gemini)
    mensagem_sem_acentos = remover_acentos(mensagem)

    policia_stems = ['assalt', 'roub', 'ameac', 'armad', 'brig']
    samu_stems = ['desmai', 'acident', 'sangr', 'infart', 'mal subit']

    for stem in policia_stems:
        if stem in mensagem_sem_acentos:
            return {
                "tipo": "POLÍCIA",
                "resposta": "Entendido! Estamos acionando a polícia imediatamente. Tente manter a calma e nos envie sua localização exata, se possível."
            }

    for stem in samu_stems:
        if stem in mensagem_sem_acentos:
            return {
                "tipo": "SAMU",
                "resposta": "Recebido! Estamos enviando a equipe médica. Por favor, envie sua localização para o atendimento."
            }

    return {
        "tipo": "INDEFINIDO",
        "resposta": "Desculpe, não consegui entender. Pode repetir a mensagem?"
    }

def eh_resposta_sim(resposta):
    respostas_sim = {'sim', 's', 'yes', 'y', 'claro', 'ok', 'okay', 'confirmar'}
    return resposta.strip().lower() in respostas_sim

def capturar_localizacao():
    print("📍 Precisamos da sua localização para enviar ajuda.")

    tentativa = input("Você conseguiu enviar a localização automática do WhatsApp? (sim/não)\n> ").strip().lower()

    if eh_resposta_sim(tentativa):
        localizacao = "Localização enviada via botão do WhatsApp"
    else:
        localizacao = input("Por favor, digite sua localização manualmente (rua, número, cidade...):\n> ").strip()

    referencia = ""
    if pergunta_sim_nao("Deseja incluir um ponto de referência?"):
        referencia = input("Digite o ponto de referência (ex: em frente ao mercado X):\n> ").strip()

    print("\n📌 Confirme os dados abaixo:")
    print(f"Localização: {localizacao}")
    if referencia:
        print(f"Ponto de referência: {referencia}")

    if not pergunta_sim_nao("Está tudo certo? "):
      print("⚠️ Localização não confirmada. Por favor, tente novamente.")
      return capturar_localizacao()

    return {
        "localizacao": localizacao,
        "referencia": referencia
    }


def simular_fluxo(mensagem):
    print(f"\n=== Nova Solicitação ===\nMensagem: {mensagem}\n")
    resultado = classificar_emergencia(mensagem)

    print(f"✅ Tipo de Atendimento: {resultado['tipo']}")
    print(f"🧠 Resposta do SOSZap: {resultado['resposta']}")

    if resultado['tipo'] == "INDEFINIDO":
        print("⚠️ Por favor, envie uma mensagem clara para que possamos ajudar.")
        return

    dados_localizacao = capturar_localizacao()

    print("\n📦 Dados completos para envio:")
    print(f"Tipo: {resultado['tipo']}")
    print(f"Mensagem: {mensagem}")
    print(f"Localização: {dados_localizacao['localizacao']}")
    if dados_localizacao['referencia']:
        print(f"Ponto de Referência: {dados_localizacao['referencia']}")

    print("=======================================\n")
    data_de_hoje = date.today().strftime("%d/%m/%Y")
    resultado = agente_solicitacao(mensagem, data_de_hoje)
    print(f"{resultado}")

    print("=== Fim da solicitação ===\n")


In [3]:
%pip -q install google-genai

# Configura a API Key do Google Gemini
import unicodedata
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

    # Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/7

In [ ]:
##########################################
# --- Agente 1: Solicitação de SOS --- #
##########################################

def agente_solicitacao(mensagem, data_de_hoje):
    solicitacao = Agent(
        name="agente_solicita_sos",
        model="gemini-2.0-flash",
        description="Agente para solicitar SOS",
        tools=[google_search],
        instruction="""
          Você é um atendente de emergência via WhatsApp.
          Está recebendo uma chamada de emergencia e precisa envia um mensagem para deixar a pessoa calma bem curta e direta.
          Também precisa informar que já encontrou em contato com as autoridade médica no caso da Samu ou força policial caso for policia e que já encaminhou ajuda.
          A resposta não pode passar de letras.
        """
       )
    entrada_do_agente_solicitacao = f"Mensagem:{mensagem}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    resultado = call_agent(solicitacao, entrada_do_agente_solicitacao)
    return resultado


In [7]:
if __name__ == "__main__":
    # Exemplo 1: Polícia
    simular_fluxo("Socorro! Tem um homem tentando me assaltar!")

    # Exemplo 2: SAMU
    #simular_fluxo("Minha mãe desmaiou e está sem respiração!")


=== Nova Solicitação ===
Mensagem: Socorro! Tem um homem tentando me assaltar!

✅ Tipo de Atendimento: POLÍCIA
🧠 Resposta do SOSZap: Entendido! Estamos acionando a polícia imediatamente. Tente manter a calma e nos envie sua localização exata, se possível.
📍 Precisamos da sua localização para enviar ajuda.
Você conseguiu enviar a localização automática do WhatsApp? (sim/não)
> sim
Deseja incluir um ponto de referência? (sim/não): sim
Digite o ponto de referência (ex: em frente ao mercado X):
> sim

📌 Confirme os dados abaixo:
Localização: Localização enviada via botão do WhatsApp
Ponto de referência: sim
Está tudo certo?  (sim/não): sim

📦 Dados completos para envio:
Tipo: POLÍCIA
Mensagem: Socorro! Tem um homem tentando me assaltar!
Localização: Localização enviada via botão do WhatsApp
Ponto de Referência: sim

Olá! Mantenha a calma. Recebi seu pedido de socorro e entendo o quão assustador deve ser essa situação. Já notifiquei a polícia e eles estão a caminho para te ajudar. Tente se